In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
import scipy
from scipy.optimize import curve_fit
from scipy import interpolate
from matplotlib import pyplot as plt
import matplotlib
from scipy import constants
from numpy.polynomial import Polynomial
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

In [ ]:
# setup basic arguments for qed-ci calculation
omega = 0.12086
# z-matrix for LiH
mol_str_LiH = """
Li
H 1 1.546938775510204
symmetry c1
"""

# options for the PFHamiltonian Generator class - include cavity effects
cavity_dict = {
    'omega_value' : omega,
    'lambda_vector' : np.array([0, 0, 0.005]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 1, #<== this is a minimal photon basis, should explore increasing this 
}

# options for PFHamiltonian Generator class - exclude cavity effects
cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [ ]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""
options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    'num_roots' : 2
}

# number of bondlengths in the scan
N_R = 1000

# number of electronic states to save
N_el = 10

# array  for energies inside the cavity
cavity_E_array_LiH = np.zeros((N_R, N_el))

r_data = np.linspace(1.0, 2.4, N_R)
psi4.set_options(options_dict)
fci_S0 = []
fci_S1 = []
r_idx_LiH = 0
for r in r_data:
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_S0.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    fci_S1.append(wfn.variable("CI ROOT 1 TOTAL ENERGY"))
    LiH_cav = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    cavity_E_array_LiH[r_idx_LiH,:] = LiH_cav.CIeigs[:N_el]
    r_idx_LiH += 1

#cavity_E_array_LiH = np.ones_like(cavity_E_array_LiH) * omega + cavity_E_array_LiH

##  Scan centered around ground state PES minimum

In [ ]:

cavity_S0 = np.ones_like(cavity_E_array_LiH[:, 0]) * omega + fci_S0


In [ ]:
##plt.plot(r_data, cavity_E_array_LiH[:, 1])
#plt.plot(r_data, cavity_E_array_LiH[:, 2], 'm', label='LP')
#plt.plot(r_data, cavity_E_array_LiH[:, 3], 'c', label='UP')
plt.plot(r_data, cavity_E_array_LiH[:, 1:])

#plt.plot(r_data, cavity_S0, 'r', label='S0')
#plt.plot(r_data, fci_S1, 'b', label='S1')
plt.legend()
plt.show()



## Scan centered around excited state PES minimum

In [ ]:
#to get more accurate measurment around minimum
mol_tmpl_e = """
Li
H 1 **R**
symmetry c1
"""

r_data_e = np.linspace(1.8, 1.9, 200)
psi4.set_options(options_dict)
fci_S0_e = []
fci_S1_e = []
for r in r_data_e:
    mol_str_e = mol_tmpl.replace("**R**", str(r))
    mol_e = psi4.geometry(mol_str_e)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_S0_e.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    fci_S1_e.append(wfn.variable("CI ROOT 1 TOTAL ENERGY"))

In [ ]:
#morse potential scan
mol_tmpl_e = """
Li
H 1 **R**
symmetry c1
"""


r_data_morse = np.linspace(1, 4, 200)
psi4.set_options(options_dict)
fci_S0_morse = []
fci_S1_morse = []
for r in r_data_morse:
    mol_str_e = mol_tmpl.replace("**R**", str(r))
    mol_e = psi4.geometry(mol_str_e)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_S0_morse.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    fci_S1_morse.append(wfn.variable("CI ROOT 1 TOTAL ENERGY"))

## Permanent Dipole Moment Calculations

In [ ]:
# z-matrix for ground state LIH
LiH_g_string = """
Li       0.0000000000      0.0000000000      0.0000000000                 
H        0.0000000000      0.0000000000      1.5459183673469388     
symmetry c1
"""

LiH_g = psi4.geometry(LiH_g_string)

psi4.set_options(options_dict)

E, wfn_g = psi4.energy('SCF', return_wfn=True,molecule=LiH_g)
psi4.oeprop(wfn, 'DIPOLE', molecule=LiH_g)

In [ ]:

# z-matrix for first excited state state LIH
LiH_e_string = """
Li       0.0000000000      0.0000000000      0.0000000000                 
H        0.0000000000      0.0000000000      1.8581632653061224 
symmetry c1
"""

options_dict1 = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    'TDSCF_STATES' : 2
}

LiH_e = psi4.geometry(LiH_e_string)

psi4.set_options(options_dict1)

#E, wfn_e = psi4.energy('TD-SCF/STO-3g', return_wfn=True, molecule=LiH_e,)

psi4.set_options(options_dict)
E, wfn_e = psi4.energy('SCF', return_wfn=True, molecule=LiH_e,)
psi4.oeprop(wfn_e, 'DIPOLE', molecule=LiH_e)

## Calculation of k
Fit ground state PES of H2 to a quintic polynomial

In [ ]:
amu_to_au = 1822.89


mA_kg = 1.00784 * (10 ** (-3) / (6.022 * 10 ** 23) )
mB_kg = 6.9410 * (10 ** (-3) / (6.022 * 10 ** 23) )
mA_au = 1.00784 * amu_to_au
mB_au = 6.9410 * amu_to_au
mu_au = (mA_au * mB_au )/ (mA_au + mB_au)
mu_kg = (mA_kg * mB_kg) / (mA_kg + mB_kg)  
print("mu_au: ", mu_au)

In [ ]:
#r_data = np.linspace(1.3, 1.95, 50)


#find crossover in cavity E array 1 and 2 and stitch them together
def find_crossover(array1, array2):

    print("array 1 - array 2, ", array1 - array2)

    idx =(np.abs(array1 - array2)).argmin()

    print(array1[idx]- array2[idx])
    return idx

index = find_crossover(cavity_E_array_LiH[:, 1], cavity_E_array_LiH[:, 2])
print(index)


UP_array_full = cavity_E_array_LiH[:, 3]
    
LP_array_1 = cavity_E_array_LiH[index:,2]
LP_array_2 = cavity_E_array_LiH[:index,1]
LP_array_full = np.concatenate((LP_array_2, LP_array_1), axis=0)


au_to_SI = (4.35974 * 10 ** (-18)) * 10 ** 20
min_S0_loc = np.argmin(fci_S0[:])
min_S1_loc = np.argmin(fci_S1_e[:])
min_LP_loc = np.argmin(LP_array_full)
min_UP_loc = np.argmin(UP_array_full)
r_eq_ang = r_data[min_S0_loc]
r_eq_au = r_eq_ang / psi4.constants.bohr2angstroms
print(f'Min on S0 is {r_data[min_S0_loc]}')
print(f'Min on S1 is {r_data_e[min_S1_loc]}')
print(f'Min on LP is {r_data[min_LP_loc]}')
print(f'Min on UP is {r_data[min_UP_loc]}')

plt.plot(r_data, UP_array_full,label='Upper polariton')
plt.plot(r_data, LP_array_full,label='Lower polariton')
plt.plot(r_data, fci_S1,label='cavity free excited')
plt.plot(r_data, fci_S0, label='cavity free ground')
plt.legend()
plt.show()



## Calculation of $ \text{x}_0 $

$$ \frac{\hbar}{2}\sqrt{\frac{k}{\mu}} = \frac{k}{2}(x_0 - x_{eq})^2 + V_0 $$

Expanded, solved for $ x_o $, and found zeros using quadratic formula

In [ ]:

r_data_meters = r_data * 10 ** -10


au_to_SI = (4.35974 * 10 ** (-18)) * 10 ** 20
min_S0_loc = np.argmin(fci_S0[:])
r_eq_ang =r_data_meters[min_S0_loc]

print(r_eq_ang)

# Fitting S0 PES to a quintic polynomial
poly = np.poly1d(np.polyfit(r_data_meters, fci_S0, 4))

poly_array = np.asarray(poly)


#Taking first and second derivative of S0 PES and evaluating at r_eq
first_derivative = poly.deriv()
second_derivative = first_derivative.deriv()
k_test= second_derivative(r_eq_ang)
k_test_SI = k_test * (4.35974 * 10 ** (-18))
print(k_test_SI)

#plotting S0 PES and quintic fit
plt.plot(r_data_meters, poly(r_data_meters), 'm-', label='fit')
plt.plot(r_data_meters, fci_S0[:], 'bo', label='cavity free |g>')
plt.show

In [ ]:
mu_amu = 1.00784 * 6.9410 / (1.00784 + 6.9410)
mu_kg = mu_amu  * 10 ** (-3) / (6.022 * 10 ** 23) 
r_eq_SI = r_eq_ang * 10 ** (-10)
h_bar = constants.hbar
V_0_loc = np.argmin(fci_S0)
V_0 = fci_S0[V_0_loc] * 4.35974 * 10 ** (-18)
left = (h_bar / 2) * np.sqrt(k_test_SI / mu_kg)
a = 0.5 * k_test_SI 
b = -k_test_SI * r_eq_SI
c = 0.5 * k_test_SI * (r_eq_SI ** 2) - left
zeros_n = (-b - np.sqrt((b ** 2) - 4 * a * c)) / (2 * a)
zeros_p = (-b + np.sqrt((b ** 2) - 4 * a * c)) / (2 * a)
x0_angstrom = zeros_n * 10 ** 10
x0_au = x0_angstrom / psi4.constants.bohr2angstroms
print(x0_angstrom)




## Calculation of the Huang-Rhys Factor
Huang Rhys factor can be calculated by both

$$ S = 1/2(\Delta x / x_0)^2 \tag{Turner}$$

from the mode anharmonicity paper

and 

$$ S = \frac{m\omega_{vib} \Delta x^2}{2 \hbar} \tag{Hsu}$$

from the polaritonic Huang-Rhys factor paper

In [ ]:



# Constants and Variables
delta_au = (r_data_e[min_S1_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_LP_au = (r_data[min_LP_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_UP_au = (r_data[min_UP_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_m = (r_data_e[min_S1_loc] - r_data[min_S0_loc]) * 10 ** (-10)
delta_angstrom = (r_data_e[min_S1_loc] - r_data[min_S0_loc])
omega_vib = np.sqrt(k_test_SI / mu_kg)
h_bar = constants.hbar
x0_test = np.sqrt(h_bar * omega_vib / k_test_SI)
x_0_au = (x0_test * 10 ** 10)  / psi4.constants.bohr2angstroms


# Turner
S_Turner = 0.5 * (delta_au / x_0_au ) ** 2


#g and LP 

HR_LP = 0.5 * (delta_LP_au / x_0_au ) ** 2
HR_UP = 0.5 * (delta_UP_au / x_0_au ) ** 2

print('|g> and |e> HR Factor')
print(S_Turner)


print('|g> and LP HR Factor')
print(HR_LP)
print('|g> and UP HR Factor')
print(HR_UP)

In [ ]:
r_data_meters = r_data * 10 ** -10


print("mu_au: ", mu_au)
r_data_au = r_data / psi4.constants.bohr2angstroms

au_to_SI = (4.35974 * 10 ** (-18)) * 10 ** 20
min_S0_loc = np.argmin(fci_S0[:])
r_eq_ang =r_data[min_S0_loc]
r_eq_au =r_data_au[min_S0_loc]


print("r_eq_au : " , r_eq_au)


# Fitting S0 PES to a quintic polynomial

#can use this line to only fit to bottom of well for cubic and harmonic
#poly = np.poly1d(np.polyfit(r_data_meters[50:100], fci_S0[50:100], 4))

poly = np.poly1d(np.polyfit(r_data_au, fci_S0, 5))

poly_array = np.asarray(poly)


#Taking first and second derivative of S0 PES and evaluating at r_eq
first_derivative = poly.deriv()
second_derivative = first_derivative.deriv()
k_au = second_derivative(r_eq_au)
print("k_au: ", k_au)

#plotting S0 PES and quintic fit
plt.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
plt.plot(r_data_au, fci_S0[:], 'bo', label='cavity free |g>')
plt.show()

angstrom_to_bohr = 1.88973
x_min = r_data_au[0]
x_max = r_data_au[-1]



hbar = 1

# number of grid points 
N = 2001
# define grid
x = np.linspace(x_min, x_max, N)


V_y = np.polyval(np.asarray(poly), (x))
print("min of v_y: ", min(V_y))



def get_fd_wfn(x, V_y: np.array, use_5_point_stencil = False):

    hbar = 1
    #r_eq_SI = r_eq_ang * 10 ** (-10)
    #r_eq_au = r_eq_ang * angstrom_to_bohr


    #kinetic enrgy in joules, need to convert to hartree
    #joule_to_hartree  =  (2.294 * 10 ** 17) 


    # define grid spacing h
    h = x[1]-x[0]

    # create arrays for T, V, and H - we truncate the smallest and largest grid points where 
    # the centered finite difference derivatives cannot be defined
    T = np.zeros((N-2, N-2))
    V = np.zeros((N-2, N-2))
    H = np.zeros((N-2, N-2))

    # this uses the 3 point stencil; we can adapt to use a 5 point and it might improve accuracy


    if not use_5_point_stencil:
        for i in range(N-2):
            for j in range(N-2):
                if i==j:
                    T[i,j]= -2
                elif np.abs(i-j)==1:
                    T[i,j]=1
                else:
                    T[i,j]=0

        T = -T *( hbar ** 2 / (2 * mu_au* h**2))
        #T =  (- (hbar ** 2) / (2* mu_kg)) *  (1 / ( h**2)) * joule_to_hartree  * T


    elif use_5_point_stencil:
        for i in range(N-2):
            for j in range(N-2):
                if i==j:
                    T[i,j]= -30
                elif np.abs(i-j)==1:
                    T[i,j]=16
                elif np.abs(i-j)==2:
                    T[i,j]=-1

        T = -T *  ((hbar ** 2) / (2* mu_au))*  (1 / ( 12 * h**2)) 


    for i in range(N-2):
        for j in range(N-2):
            if i==j:
                V[i,j]= V_y[i+1]
            else:
                V[i,j]=0
                
    H = T + V

    #print((-T * hbar ** 2 / (2 * mu_kg* h**2)) * (2.294 * 10 ** 17))
    #print(V)

    vals, vecs = np.linalg.eigh(H)

    if np.average(vecs[:, 0]) < 0:
        vecs = vecs * -1

    return vals, vecs

vals, vecs = get_fd_wfn(x, V_y, use_5_point_stencil=True)

print(vals[0])

vals, vecs = get_fd_wfn(x, V_y)
print(vals[0])
print(vals[1])
print(vals[2])


plt.plot(x[1:N-1], vecs[:,0], 'r-')
plt.show()



def calculate_x_0(x_array, V_array, E):
    #print(V_array)
    #find bototm of well
    min = np.argmin(V_array)
    #chop off right side
    V_array = V_array[:min]
    idx =(np.abs(V_array- E)).argmin()
    return np.abs(x[idx] - r_eq_ang)
x_0 = calculate_x_0(x, V_y, vals[0])
print("x0: ", x_0)





# Constants and Variables
delta_au = (r_data_e[min_S1_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_LP_au = (r_data[min_LP_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_UP_au = (r_data[min_UP_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_m = (r_data_e[min_S1_loc] - r_data[min_S0_loc]) * 10 ** (-10)
delta_angstrom = (r_data_e[min_S1_loc] - r_data[min_S0_loc])
omega_vib = np.sqrt(k_au/ mu_au)
#h_bar = constants.hbar
#x0_test = np.sqrt(h_bar * omega_vib / k_test_SI)

#x0 calculated in au
x_0_au = x_0



# Turner
S_Turner = 0.5 * (delta_au / x_0_au ) ** 2


#g and LP 

HR_LP = 0.5 * (delta_LP_au / x_0_au ) ** 2
HR_UP = 0.5 * (delta_UP_au / x_0_au ) ** 2


print("X_0: ", x_0_au)
print('|g> and |e> HR Factor')
print(S_Turner)


print('|g> and LP HR Factor')
print(HR_LP)
print('|g> and UP HR Factor')
print(HR_UP)


## Polaritonic Huang-Rhys Factor

$$ S_{pol} = \frac{|\Delta\mu \epsilon_s|^2}{2\epsilon_0 V_{eff,s} \hbar \omega_s} \tag{Hsu} $$
Where:

$\Delta\mu \equiv \text{Permanant Dipole Difference} = \mu_g - \mu_e$

$\epsilon_s \equiv \text{Unit Vector}$

$V_{eff,s} \equiv \text{Effective Mode Volume}$

$\omega_s \equiv \text{Frequency of a single photonic mode}$


In [ ]:
mu_g = 1.9066253 * 3.335 * 10 ** (-30)
mu_e = 1.9450498 * 3.335 * 10 ** (-30)
d_mu_squared = (mu_g - mu_e) ** 2
epsilon_s = 1
epsilon_0 = constants.epsilon_0
lambda_mag = 0.05 #ask about units
V_eff = h_bar / (epsilon_0 * lambda_mag ** 2)  
omega_s = 0.12086  #ask about units
S_pol = d_mu_squared / (2 * epsilon_0 * V_eff * h_bar * omega_s)
print(S_pol)

First plot the ground-state potential energy surfaces for $ \text{LiH} $ inside and outisde the cavity.  The effect of the cavity will raise the energy slightly.

In [ ]:


fci_cavity_S0 = np.ones_like(cavity_E_array_LiH[:, 0]) * omega + fci_S0



plt.plot(r_data, fci_cavity_S0, 'r-', label='cavity free |g + $\omega$>')
plt.plot(r_data, fci_S1, 'b-', label='cavity free |e>')
plt.plot(r_data, LP_array_full, 'm-', label='cavity LiH |LP>')
#plt.plot(r_data, cavity_E_array_LiH[:,1], 'g-', label='cavity LiH |TRIPLET>')
plt.plot(r_data, cavity_E_array_LiH[:,3], 'c-', label='cavity LiH |UP>')
#plt.xlim(1.2, 1.75)
#plt.ylim(-7.76, -7.72)
plt.legend()
plt.show()


In [ ]:
#FCF calculator


def get_fcf_matrix(potential_1, potential_2, matrix_size = 6, potential_1_is_groundstate = False):

    #geneerate 2 sets of wavefunctions for the potentials

    r_data_au = r_data / psi4.constants.bohr2angstroms

    min_potential_1_loc = np.argmin(potential_1[:])
    r_eq_au =r_data_au[min_potential_1_loc]

    print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    poly = np.poly1d(np.polyfit(r_data_au, potential_1, 5))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    print("k_au: ", k_au)

    #plotting S0 PES and quintic fit
    plt.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    plt.plot(r_data_au, potential_1, 'bo', label='cavity free |g>')
    plt.show()

    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)

    V_y = np.polyval(np.asarray(poly), (x))


    vals1, vecs1 = get_fd_wfn(x, V_y, use_5_point_stencil=True)
    #vals1, vecs1 = get_fd_wfn(x, V_y)



    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('bond length (m)')
    ax1.set_ylabel('wfn', color=color)
    ax1.plot(x[1:N-1], vecs1[:,0], 'r')
    ax1.plot(x[1:N-1], vecs1[:,1], 'b', )
    ax1.plot(x[1:N-1], vecs1[:,2], 'g')
    ax1.plot(x[1:N-1], vecs1[:,3], 'y')
    ax1.tick_params(axis='y', labelcolor=color)


    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    ax2.plot(r_data_au, potential_1, 'bo', label='state_1')
    ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()


    min_potential_2_loc = np.argmin(potential_2[:])
    r_eq_au =r_data_au[min_potential_2_loc]

    print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    #can use this line to only fit to bottom of well for cubic and harmonic
    #poly = np.poly1d(np.polyfit(r_data_meters[50:100], fci_S0[50:100], 4))

    poly = np.poly1d(np.polyfit(r_data_au, potential_2, 5))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    print("k_au: ", k_au)

    #plotting S0 PES and quintic fit
    plt.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    plt.plot(r_data_au, potential_2, 'bo', label='cavity free |g>')
    plt.show()

    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)

    V_y = np.polyval(np.asarray(poly), (x))


    vals2, vecs2 = get_fd_wfn(x, V_y, use_5_point_stencil=True)
    #vals2, vecs2 = get_fd_wfn(x, V_y)



    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('bond length')
    ax1.set_ylabel('wfn', color=color)
    ax1.plot(x[1:N-1], vecs2[:,0], 'r')
    ax1.plot(x[1:N-1], vecs2[:,1], 'b')
    ax1.plot(x[1:N-1], vecs2[:,2], 'g')
    ax1.plot(x[1:N-1], vecs2[:,3], 'y')

    print(vals2[0])
    print(vals2[1])
    print(vals2[2])
    print(vals2[3])

    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    ax2.plot(r_data_au, potential_2, 'bo', label='state_1')
    ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()


    FCF_matrix = np.zeros((matrix_size,matrix_size))

    for i in range(FCF_matrix.shape[0]):
        for j in range(FCF_matrix.shape[0]):

            FCF_matrix[i][j] = np.trapz(vecs1[:,i] * vecs2[:,j]) 
            FCF = np.absolute(FCF_matrix) ** 2 

    return FCF


fcf = get_fcf_matrix(fci_S1, UP_array_full, 6)
#fcf = get_fcf_matrix(LP_array_full, UP_array_full, 6)
print(fcf)

import pandas as pd
pd.DataFrame(fcf).to_clipboard()


In [ ]:
#morse potential scan
"""
options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    'num_roots' : 2
}

r_data_morse = np.linspace(1, 5, 1000)
psi4.set_options(options_dict)
fci_S0_morse = []
fci_S1_morse = []
for r in r_data_morse:
    mol_str_e = mol_tmpl.replace("**R**", str(r))
    mol_e = psi4.geometry(mol_str_e)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_S0_morse.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    fci_S1_morse.append(wfn.variable("CI ROOT 1 TOTAL ENERGY"))
"""

In [ ]:
import copy
fci_s0_morse_copy = copy.deepcopy(fci_S0_morse)

In [ ]:
#not working yet
"""
#morse potential scan

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    'num_roots' : 2
}

r_data_morse = np.linspace(1, 5, 1000)
psi4.set_options(options_dict)
fci_S0_morse = []
fci_S1_morse = []
for r in r_data_morse:
    mol_str_e = mol_tmpl.replace("**R**", str(r))
    mol_e = psi4.geometry(mol_str_e)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_S0_morse.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    fci_S1_morse.append(wfn.variable("CI ROOT 1 TOTAL ENERGY"))
"""
#import copy
#fci_S0_morse = fci_s0_morse_copy
#fci_s0_morse_copy = copy.deepcopy(fci_S0_morse)
#fci_S0_morse = list(np.array(fci_S0_morse) - min(fci_S0_morse))

angstrom_to_bohr = 1.88973

De_au =max(fci_S0_morse) - min(fci_S0_morse)


r_data_morse_meters = r_data_morse * 10 ** -10
r_data_morse_au = r_data_morse * angstrom_to_bohr


min_fci_morse_loc = np.array(fci_S0_morse).argmin()
r_eq_SI =r_data_morse_meters [min_fci_morse_loc ]
r_eq_au = r_data_morse_au [min_fci_morse_loc ]
print("r_eq_au: ", r_eq_au)


# Fitting S0 PES to a quintic polynomial
poly = np.poly1d(np.polyfit(r_data_morse_au , fci_S0_morse, 12))

poly_array = np.asarray(poly)

#Taking first and second derivative of S0 PES and evaluating at r_eq
first_derivative = np.polyder(poly)
second_derivative = np.polyder(poly, 2)
k_morse_au = second_derivative(r_eq_au)


print("k_morse_au: ", k_morse_au)

#plotting S0 PES and quintic fit
plt.plot(r_data_morse_au , poly(r_data_morse_au ), 'm-', label='fit')
plt.plot(r_data_morse_au , fci_S0_morse[:], 'bo', label='cavity free |g>')
plt.show()






import numpy as np
from scipy.constants import h, hbar, c, u
from scipy.special import factorial
from scipy.special import genlaguerre, gamma

FAC = 100 * h * c

hbar = 1
h = 2 * np.pi

class Morse:
    """A class representing the Morse oscillator model of a diatomic."""

    def __init__(self, mA, mB, we, wexe, re, Te, De, ke, a):
        """Initialize the Morse model for a diatomic molecule.

        mA, mB are the atom masses (atomic mass units).
        we, wexe are the Morse parameters (cm-1).
        re is the equilibrium bond length (m).
        Te is the electronic energy (minimum of the potential well; origin
            of the vibrational state energies).

        """
        #print(u)
        self.mA, self.mB = mA, mB
        self.mu = mA*mB/(mA+mB)
        self.we, self.wexe = we, wexe
        self.re = re
        self.Te = Te
        print(Te)

        self.De = De
        self.ke = ke
        #self.ke = (2 * np.pi * c * 100 * we)**2 * self.mu
        #  Morse parameters, a and lambda.
        self.a = a
        print(self.a)
        self.lam = np.sqrt(2 * self.mu * self.De) / (self.a * hbar)
        # Maximum vibrational quantum number.
        self.vmax = int(np.floor(self.lam - 0.5))

        self.make_rgrid()
        self.V = self.Vmorse(self.r)

    def make_rgrid(self, n=len(fci_S0_morse), rmin=None, rmax=None, retstep=False):
        """Make a suitable grid of internuclear separations."""

        self.rmin, self.rmax = rmin, rmax
        if rmin is None:
            # minimum r where V(r)=De on repulsive edge
            self.rmin = self.re - np.log(2) / self.a
        if rmax is None:
            # maximum r where V(r)=f.De
            f = 0.999
            self.rmax = self.re - np.log(1-f)/self.a
        self.r, self.dr = np.linspace(self.rmin, self.rmax, n,
                                      retstep=True)
        if retstep:
            return self.r, self.dr
        return self.r



    def Vmorse(self, r):
        """Calculate the Morse potential, V(r).

        Returns the Morse potential at r (in m) for parameters De
        (in J), a (in m-1) and re (in m).

        """

        return self.De * (1 - np.exp(-self.a*(r - self.re)))**2


    def calc_psi(self, v, r=None, normed=True, psi_max=1):
        """Calculates the Morse oscillator wavefunction, psi_v.

        Returns the Morse oscillator wavefunction at vibrational
        quantum number v. The returned function is "normalized" to
        give peak value psi_max.

        """

        if r is None:
            r = self.r
        z = 2 * self.lam * np.exp(-self.a*(r - self.re))
        alpha = 2*(self.lam - v) - 1
        psi = (z**(self.lam-v-0.5) * np.exp(-z/2) *
               genlaguerre(v, alpha)(z))
        psi *= psi_max / np.max(psi)
        return psi

    def calc_psi_z(self, v, z):
        alpha = 2*(self.lam - v) - 1
        psi = (z**(self.lam-v-0.5) * np.exp(-z/2) *
               genlaguerre(v, alpha)(z))
        Nv = np.sqrt(factorial(v) * (2*self.lam - 2*v - 1) /
                     gamma(2*self.lam - v))
        return Nv * psi

    def plot_V(self):
        """Plot the Morse potential on Axes ax."""

        plt.plot(self.r, self.V)
        plt.show()

    def get_vmax(self):
        """Return the maximum vibrational quantum number."""
        return int(self.we / 2 / self.wexe - 0.5)


    def calc_energy(self, v):

        return self.we * (v +.5) - self.wexe*(v+0.5)**2



amu_to_au = 1822.89


mA_kg = 1.00784 * (10 ** (-3) / (6.022 * 10 ** 23) )
mB_kg = 6.9410 * (10 ** (-3) / (6.022 * 10 ** 23) )
mA_au = 1.00784 * amu_to_au
mB_au = 6.9410 * amu_to_au
mu_au = (mA_au * mB_au )/ (mA_au + mB_au)
mu_kg = (mA_kg * mB_kg) / (mA_kg + mB_kg)  
print("mu_au: ", mu_au)

#D_e = 0.10017924832587166 * psi4.constants.hartree2J
De_au = 0.10017924832587166 
T_e = min(fci_S0_morse)

#morse parameters
#a = np.sqrt(k_morse_au / (2 * De_au))
a = np.sqrt((k_morse_au * mu_au)  / hbar)

print("a : ", a)


#omega_e = (a / (2 * np.pi * constants.speed_of_light)) * np.sqrt((2 * D_e)/ mu_kg) * FAC
#omega_e  = (a / (2 * np.pi * 137)) * np.sqrt((2 * De_au)/ mu_au)
#print("we: ", omega_e)
omega_e  = np.sqrt(k_morse_au/mu_au)


omega_e_x_e = ( omega_e **2) / (4 * De_au)


print("a : ", a)

print("we: ", omega_e)
print("wexe: ", omega_e_x_e)


X = Morse(mA_au, mB_au, omega_e, omega_e_x_e, r_eq_au, T_e, De_au, k_morse_au, a)
X.make_rgrid()
X.V = X.Vmorse(X.r) 


X.plot_V()


psi_0 = X.calc_psi(0)
#psi_1 = X.calc_psi(1)


E_0 = X.calc_energy(0)
print("E_0:", E_0 + fci_s0_morse_copy[min_fci_morse_loc])


E_1 = X.calc_energy(1)
print("E_1:", E_1 + fci_s0_morse_copy[min_fci_morse_loc])

E_2 = X.calc_energy(2)
print("E_2:", E_2+ fci_s0_morse_copy[min_fci_morse_loc])


plt.plot( r_data_morse_au , psi_0)
#plt.plot( r_data_morse_au , psi_1)
plt.show()




def calculate_x_0(x_array, V_array, E):
    #print(V_array)
    #find bototm of well
    min = np.argmin(V_array)
    #chop off right side
    V_array = V_array[:min]
    idx =(np.abs(V_array- E)).argmin()
    return np.abs(x[idx] - r_eq_ang)


x_0 = calculate_x_0(r_data_morse_au, X.V ,  E_0)





omega_vib = np.sqrt(k_morse_au / mu_au)
h_bar = 1
#x0_test = np.sqrt(h_bar * omega_vib / k_test_SI)
 
#x0 calculated in meters, convert to angstroms then to bohr, AU



# Turner
S_Turner = 0.5 * (delta_au / x_0_au ) ** 2


#g and LP 

HR_LP = 0.5 * (delta_LP_au / x_0_au ) ** 2
HR_UP = 0.5 * (delta_UP_au / x_0_au ) ** 2


print("X_0: ", x_0_au)
print('|g> and |e> HR Factor')
print(S_Turner)


print('|g> and LP HR Factor')
print(HR_LP)
print('|g> and UP HR Factor')
print(HR_UP)

